In [1]:
import os
import numpy as np

In [2]:
!nvidia-smi

Mon Sep  6 19:55:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| N/A   55C    P0    31W /  N/A |    588MiB /  7973MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
PATH_TO_YOLO = "/app/yolov5"
PATH_TO_WEIGHTS = "/app/mmk_smoke_detection/notebooks/fire-detection/fire-detection-from-images/pytorch/object-detection/yolov5/experiment1/best.pt"

In [4]:
from preparator import ImageDetectionController
from fire_detector import FireDetector

In [5]:
detection_controller = ImageDetectionController()

In [6]:
from classJson import JsonData

PATH_TO_IMAGES = "/app/mmk_smoke_detection/flow_data/172.30.71.109"
CONFIG_PATH = "./conf/conf_ct131405.json"

conf = JsonData(name=CONFIG_PATH)
data_matrix = conf.return_matrix()
matrix = data_matrix["matrix"]
max_height = data_matrix["maxHeight"]
max_width = data_matrix["maxWidth"]

In [7]:
from preparator import ImageDrawer

image_drawer = ImageDrawer(
    detection_controller=detection_controller,
    matrix=matrix,
    max_height=max_height,
    max_width=max_width
)

In [8]:
fire_detector = FireDetector(
    path_to_yolo_v5=PATH_TO_YOLO,
    path_to_weights=PATH_TO_WEIGHTS,
    detection_controller=detection_controller
)

/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554793803/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
YOLOv5 🚀 v5.0-407-g5487451 torch 1.8.1 CPU

Fusing layers... 
Model Summary: 232 layers, 7246518 parameters, 0 gradients
Adding AutoShape... 


In [9]:
import glob

images = glob.glob(os.path.join(PATH_TO_IMAGES, "*"))
images = images[:1000]

for image in images:
    fire_detector.handle_image(
        matrix=matrix,
        max_width=max_width,
        max_height=max_height,
        image_path=image
    )

In [10]:
image_drawer._detection_controller._detections_of_image

{'/app/mmk_smoke_detection/flow_data/172.30.71.109/frame_08-25-2021_10:35:59.jpg': [{'place': '',
   'detection': [585, 0, 693, 87],
   'score': 0.27695488929748535,
   'label': 'fire',
   'label_idx': 2,
   'line_thickness': 4},
  {'place': '',
   'detection': [581, 0, 887, 172],
   'score': 0.2592630386352539,
   'label': 'fire',
   'label_idx': 2,
   'line_thickness': 4}],
 '/app/mmk_smoke_detection/flow_data/172.30.71.109/frame_08-26-2021_11:32:59.jpg': [{'place': '',
   'detection': [3, 24, 475, 199],
   'score': 0.30643001198768616,
   'label': 'fire',
   'label_idx': 2,
   'line_thickness': 4},
  {'place': '',
   'detection': [484, 21, 854, 193],
   'score': 0.25722384452819824,
   'label': 'fire',
   'label_idx': 2,
   'line_thickness': 4}],
 '/app/mmk_smoke_detection/flow_data/172.30.71.109/frame_08-26-2021_16:01:43.jpg': [{'place': '',
   'detection': [661, 41, 734, 114],
   'score': 0.48542869091033936,
   'label': 'fire',
   'label_idx': 2,
   'line_thickness': 4}],
 '/app/

In [11]:
TEST_PATH = "./fire_test_images"
image_drawer.draw_random_images(number=100, dir_to_save=TEST_PATH, use_normalized_coordinates=False)

In [12]:
PATH_TO_SAVED_FIRE_DETECTIONS_JSON = "fire_detections.json"
detection_controller.save(os.path.join(TEST_PATH, PATH_TO_SAVED_FIRE_DETECTIONS_JSON))